# Data and Library Importing

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

from sklearn.metrics import confusion_matrix,classification_report,f1_score

import time
import optuna
from optuna.integration import TFKerasPruningCallback
from optuna.visualization import plot_intermediate_values
from optuna.visualization import plot_optimization_history
from optuna.visualization import plot_param_importances
from optuna.visualization import plot_contour
from optuna.visualization import plot_pareto_front

from tensorflow.keras import Sequential
from tensorflow.keras.backend import clear_session
from tensorflow.keras.layers import Input,Dense,concatenate,Dropout,Conv1D,Bidirectional,Embedding,LSTM,SimpleRNN,GlobalAveragePooling1D
from tensorflow.keras.models import Model,load_model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import plot_model
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.metrics import CategoricalCrossentropy
from tensorflow.random import set_seed

In [2]:
set_seed(123)
np.random.seed(123)

In [3]:
label_latih=np.load('../input/smsa-dataset/label_latih.npy', allow_pickle=True)
label_validasi=np.load('../input/smsa-dataset/label_validasi.npy', allow_pickle=True)
label_uji=np.load('../input/smsa-dataset/label_uji.npy', allow_pickle=True)

In [4]:
embedding_dim = 300
max_length = 96
trunc_type = 'post'
padding_type = 'post'
oov_tok = "<OOV>"

In [5]:
teks_latih_pad=np.load('../input/smsa-dataset/teks_latih_pad.npy', allow_pickle=True)
teks_latih=np.load('../input/smsa-dataset/teks_latih.npy', allow_pickle=True)
teks_validasi_pad=np.load('../input/smsa-dataset/teks_validasi_pad.npy', allow_pickle=True)
teks_validasi=np.load('../input/smsa-dataset/teks_validasi.npy', allow_pickle=True)
teks_uji_pad=np.load('../input/smsa-dataset/teks_uji_pad.npy', allow_pickle=True)
teks_uji=np.load('../input/smsa-dataset/teks_uji.npy', allow_pickle=True)

# Modelling

In [6]:
jumlah_token=17274

In [7]:
def create_model(trial):
  lstm_units=np.zeros(2, dtype=int)
  lstm_units[0]=trial.suggest_int("lstm_units_L1", 32, 160)
  lstm_units[1]=trial.suggest_int("lstm_units_L2", 16, 80)
  do1=trial.suggest_float("dropout_L1",0,0.5)
  do2=trial.suggest_float("dropout_L2",0,0.5)
  lr=trial.suggest_float("learning_rate",1e-5,1e-2)

  inputs=Input(shape=(max_length,))
  layers=Embedding(jumlah_token, embedding_dim,input_length=max_length,trainable=False)(inputs)
  layers=Bidirectional(LSTM(lstm_units[0], return_sequences=True))(layers)
  layers=Dropout(do1)(layers)
  layers=Bidirectional(LSTM(lstm_units[1]))(layers)
  layers=Dropout(do2)(layers)
  layers=Dense(32, activation='relu')(layers)
  output=Dense(3, activation='softmax')(layers)

  model=Model(inputs=[inputs],outputs=[output])

  model.compile(optimizer=Adam(learning_rate=lr),loss="sparse_categorical_crossentropy",metrics=["accuracy"])

  return model

## Bayesian Optimization (F1-score)

In [8]:
#Fungsi Objektif
def objective_fun_f1(trial):
  model=create_model(trial)

  start=time.time()
  history=model.fit(teks_latih_pad,label_latih,epochs=25,batch_size=16,callbacks=[TFKerasPruningCallback(trial, "val_loss")],
                    validation_data=(teks_validasi_pad,label_validasi),verbose=0)
  end=time.time()

  Label_pred=model.predict(teks_uji_pad)
  label_pred=Label_pred.argmax(axis=1)
  fscore=f1_score(label_uji, label_pred, average='macro')
  duration=end-start
  model.save('Bayesian Optimization (F1-score) Model-{}.h5'.format(trial.number))

  print("F1-Score:",fscore)
  print("Duration:",duration)

  return fscore

In [9]:
study3=optuna.create_study(study_name="BO (F1-score)",
                          direction="maximize", 
                          sampler=optuna.samplers.TPESampler(), 
                          pruner=optuna.pruners.NopPruner())
study3.optimize(objective_fun_f1,n_trials=25)

[I 2023-03-10 14:36:32,906] A new study created in memory with name: BO (F1-score)
[I 2023-03-10 14:43:43,275] Trial 0 finished with value: 0.6894261723058914 and parameters: {'lstm_units_L1': 149, 'lstm_units_L2': 69, 'dropout_L1': 0.27794744011620526, 'dropout_L2': 0.18428592461908538, 'learning_rate': 0.007478259135141676}. Best is trial 0 with value: 0.6894261723058914.


F1-Score: 0.6894261723058914
Duration: 423.9984059333801


[I 2023-03-10 14:50:56,493] Trial 1 finished with value: 0.6889106702057979 and parameters: {'lstm_units_L1': 102, 'lstm_units_L2': 79, 'dropout_L1': 0.11526711221444852, 'dropout_L2': 0.45667216733419336, 'learning_rate': 0.009548565201219135}. Best is trial 0 with value: 0.6894261723058914.


F1-Score: 0.6889106702057979
Duration: 430.8779573440552


[I 2023-03-10 14:57:51,556] Trial 2 finished with value: 0.7620049450471406 and parameters: {'lstm_units_L1': 67, 'lstm_units_L2': 45, 'dropout_L1': 0.29141506109238857, 'dropout_L2': 0.16483203934860918, 'learning_rate': 0.004706561946370922}. Best is trial 2 with value: 0.7620049450471406.


...

F1-Score: 0.7283817313711576
Duration: 389.1451575756073


[I 2023-03-10 17:17:29,870] Trial 23 finished with value: 0.7199317738791423 and parameters: {'lstm_units_L1': 68, 'lstm_units_L2': 24, 'dropout_L1': 0.38381384701725174, 'dropout_L2': 0.06354990536287422, 'learning_rate': 0.004953083554699265}. Best is trial 2 with value: 0.7620049450471406.


F1-Score: 0.7199317738791423
Duration: 385.20514941215515


[I 2023-03-10 17:23:41,171] Trial 24 finished with value: 0.6623166159772533 and parameters: {'lstm_units_L1': 46, 'lstm_units_L2': 27, 'dropout_L1': 0.4385758493973493, 'dropout_L2': 0.013964509304263545, 'learning_rate': 0.008402388931434412}. Best is trial 2 with value: 0.7620049450471406.


F1-Score: 0.6623166159772533
Duration: 368.1789903640747


In [10]:
plot_optimization_history(study3)

In [11]:
plot_intermediate_values(study3)

In [12]:
plot_param_importances(study3)

In [13]:
print("Best trial:")
trial = study3.best_trial

print("  Value: ", trial.value)

print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))

Best trial:
  Value:  0.7620049450471406
  Params: 
    lstm_units_L1: 67
    lstm_units_L2: 45
    dropout_L1: 0.29141506109238857
    dropout_L2: 0.16483203934860918
    learning_rate: 0.004706561946370922


## Bayesian Optimization (Time)

In [14]:
#Fungsi Objektif
def objective_fun_time(trial):
  model=create_model(trial)

  start=time.time()
  history=model.fit(teks_latih_pad,label_latih,epochs=25,batch_size=16,callbacks=[TFKerasPruningCallback(trial, "val_loss")],
                    validation_data=(teks_validasi_pad,label_validasi),verbose=0)
  end=time.time()

  Label_pred=model.predict(teks_uji_pad)
  label_pred=Label_pred.argmax(axis=1)
  fscore=f1_score(label_uji, label_pred, average='macro')
  duration=end-start
  model.save('Bayesian Optimization (Time) Model-{}.h5'.format(trial.number))

  print("F1-Score:",fscore)
  print("Duration:",duration)

  return duration

In [15]:
study4=optuna.create_study(study_name="BO (Time)",
                          direction="minimize", 
                          sampler=optuna.samplers.TPESampler(), 
                          pruner=optuna.pruners.NopPruner())
study4.optimize(objective_fun_time,n_trials=25)

[I 2023-03-10 17:23:42,047] A new study created in memory with name: BO (Time)
[I 2023-03-10 17:29:55,683] Trial 0 finished with value: 370.47402238845825 and parameters: {'lstm_units_L1': 42, 'lstm_units_L2': 35, 'dropout_L1': 0.42625460738629506, 'dropout_L2': 0.2530460629434462, 'learning_rate': 0.004795717006006546}. Best is trial 0 with value: 370.47402238845825.


F1-Score: 0.7004723893145798
Duration: 370.47402238845825


[I 2023-03-10 17:37:01,556] Trial 1 finished with value: 423.47449350357056 and parameters: {'lstm_units_L1': 126, 'lstm_units_L2': 35, 'dropout_L1': 0.3781147386565318, 'dropout_L2': 0.23296682508731742, 'learning_rate': 0.002464031926453974}. Best is trial 0 with value: 370.47402238845825.


F1-Score: 0.646233486219876
Duration: 423.47449350357056


[I 2023-03-10 17:44:07,926] Trial 2 finished with value: 422.67722392082214 and parameters: {'lstm_units_L1': 104, 'lstm_units_L2': 59, 'dropout_L1': 0.25348996854888994, 'dropout_L2': 0.17417736950318147, 'learning_rate': 0.0017157841436678946}. Best is trial 0 with value: 370.47402238845825.


...

F1-Score: 0.6274607557396329
Duration: 364.654958486557


[I 2023-03-10 20:03:37,062] Trial 23 finished with value: 366.99341464042664 and parameters: {'lstm_units_L1': 46, 'lstm_units_L2': 16, 'dropout_L1': 0.42555363423001524, 'dropout_L2': 0.3648775943247589, 'learning_rate': 0.002642267569815532}. Best is trial 14 with value: 362.5420253276825.


F1-Score: 0.6244105071034255
Duration: 366.99341464042664


[I 2023-03-10 20:09:49,814] Trial 24 finished with value: 369.1022710800171 and parameters: {'lstm_units_L1': 61, 'lstm_units_L2': 31, 'dropout_L1': 0.4692814340462357, 'dropout_L2': 0.30077303768549535, 'learning_rate': 0.0008680980439254513}. Best is trial 14 with value: 362.5420253276825.


F1-Score: 0.5633042924383406
Duration: 369.1022710800171


In [16]:
plot_optimization_history(study4)

In [17]:
plot_intermediate_values(study4)

In [18]:
plot_param_importances(study4)

In [19]:
print("Best trial:")
trial = study4.best_trial

print("  Value: ", trial.value)

print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))

Best trial:
  Value:  362.5420253276825
  Params: 
    lstm_units_L1: 35
    lstm_units_L2: 48
    dropout_L1: 0.49445043972204883
    dropout_L2: 0.44282311777744643
    learning_rate: 0.0016492348874942958
